In [1]:
#importing all te req lib
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing import image
import numpy as np

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

#defining the data directories
train_data_dir= 'data/train'
validation_data_dir= 'data/validation'
n_training_sample= 1580
n_validation_sample= 385
epochs=10
batch_size=64


In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1580 images belonging to 4 classes.
Found 385 images belonging to 4 classes.


In [5]:
#defining the model
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(8, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(4, activation=tf.nn.softmax)])

In [6]:
#defining the optimizer and metrics
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])


In [7]:
model.fit_generator(
    train_generator,
    steps_per_epoch=n_training_sample // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=n_validation_sample // batch_size)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
24/24 [==============================] - 369s 15s/step - loss: 11.3269 - accuracy: 0.2657 - val_loss: 11.5429 - val_accuracy: 0.2839
Epoch 2/10
24/24 [==============================] - 307s 13s/step - loss: 11.9312 - accuracy: 0.2608 - val_loss: 11.5429 - val_accuracy: 0.2839
Epoch 3/10
24/24 [==============================] - 289s 12s/step - loss: 12.0046 - accuracy: 0.2503 - val_loss: 11.5429 - val_accuracy: 0.2839
Epoch 4/10
24/24 [==============================] - 259s 11s/step - loss: 12.0748 - accuracy: 0.2394 - val_loss: 11.5429 - val_accuracy: 0.2839
Epoch 5/10
24/24 [==============================] - 271s 11s/step - loss: 12.1411 - accuracy: 0.2466 - val_loss: 11.5429 - val_accuracy: 0.2839
Epoch 6/10
24/24 [==============================] - 242s 10s/step - loss: 11.7329 - accuracy: 0.2893 - val_loss: 11.5429 - val_accuracy: 0.2839
Epoch 7/10
24/24 [========================

In [8]:
#testing the model
pred= image.load_img('data/test/test/1_2.jpg', target_size=(150,150))
pred=image.img_to_array(pred)
pred= np.expand_dims(pred, axis=0)
result= model.predict(pred)[0]
classes = validation_generator.class_indices
predicted_class = None
for (c,i) in classes.items():
    if(result[i] == 1):
        predicted_class = c
        break
print('Model predicted ' + str(predicted_class))

Model predicted 3


In [9]:
print(model.predict(pred)) 

[[0. 0. 1. 0.]]
